In [1]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [2]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
print(game)
print(game.board_value())

+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |  2   |      |
+------+------+------+------+
4


In [3]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

In [4]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [10]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

0
+------+------+------+------+------+
|  16  |  4   | 256  |  2   |  4   |
+------+------+------+------+------+
|  64  | 256  |  4   | 512  |      |
+------+------+------+------+------+
|  4   |  32  |  16  |  32  | 512  |
+------+------+------+------+------+
|  4   |  32  |  4   |      |      |
+------+------+------+------+------+
| 512  |  2   | 2048 |      |      |
+------+------+------+------+------+
1
5119592.0
64692408141.0953
1
+------+------+------+------+------+
|  4   |  4   | 512  |  16  |  2   |
+------+------+------+------+------+
|  16  |  32  |  2   |  8   |      |
+------+------+------+------+------+
|  4   | 2048 |  32  |      |      |
+------+------+------+------+------+
|  64  |  2   |      |      |      |
+------+------+------+------+------+
|  4   |      |      |      |      |
+------+------+------+------+------+
1
4463244.0
219613702881.0567
2
+------+------+------+------+------+
|  4   |      |      |      |      |
+------+------+------+------+------+
|  16  |  

19
+------+------+------+------+------+
|  2   | 128  | 512  |  2   | 2048 |
+------+------+------+------+------+
| 512  | 512  |  8   |  8   | 512  |
+------+------+------+------+------+
|  64  |  16  |  2   |  4   |      |
+------+------+------+------+------+
|  2   |      |  32  |      | 512  |
+------+------+------+------+------+
|  8   |      |      |      |      |
+------+------+------+------+------+
1
5527008.0
262174538654.67288
20
+------+------+------+------+------+
|  4   | 2048 |  4   |  4   |  32  |
+------+------+------+------+------+
|  32  |  8   |  32  |  2   |  2   |
+------+------+------+------+------+
|  4   |      |  8   |  4   |      |
+------+------+------+------+------+
|      |      |      |  8   |      |
+------+------+------+------+------+
|      |      |      |      |      |
+------+------+------+------+------+
1
4197656.0
224721037447.97134
21
+------+------+------+------+------+
|  2   | 512  | 2048 |      |      |
+------+------+------+------+------+
|  2

38
+------+------+------+------+------+
|  2   |  4   |  2   | 128  |  4   |
+------+------+------+------+------+
| 256  |  16  | 256  |  4   | 128  |
+------+------+------+------+------+
| 2048 |  2   | 128  |  16  |  16  |
+------+------+------+------+------+
|      |  4   |  4   |  32  |  32  |
+------+------+------+------+------+
|      |      |  2   |  64  |      |
+------+------+------+------+------+
1
4381536.0
139599454724.62433
39
+------+------+------+------+------+
|  2   | 2048 |  64  |  2   |  8   |
+------+------+------+------+------+
|  8   | 1024 |  32  |  8   |  2   |
+------+------+------+------+------+
|  2   |      | 256  |  2   |  8   |
+------+------+------+------+------+
|      | 1024 |      |  16  |  64  |
+------+------+------+------+------+
|      |      |      |  8   |      |
+------+------+------+------+------+
1
6366804.0
132789726175.06836
40
+------+------+------+------+------+
|  16  |      |  2   |      |      |
+------+------+------+------+------+
|  8

57
+------+------+------+------+------+
|      |  2   |  8   |  2   |      |
+------+------+------+------+------+
|      | 128  |  64  |  4   |  8   |
+------+------+------+------+------+
| 2048 |  2   |  4   | 128  |  2   |
+------+------+------+------+------+
|  8   |  4   |  16  |  16  |  4   |
+------+------+------+------+------+
|  2   |  32  |  4   |  2   |  8   |
+------+------+------+------+------+
1
4233064.0
59585146603.81904
58
+------+------+------+------+------+
| 1024 |  2   |  4   |  16  |  8   |
+------+------+------+------+------+
|  2   | 128  |  8   |  4   | 512  |
+------+------+------+------+------+
| 1024 |  4   | 128  |  2   |  8   |
+------+------+------+------+------+
|      |  2   |  4   |  8   |  32  |
+------+------+------+------+------+
|      |      | 2048 |  4   | 1024 |
+------+------+------+------+------+
1
7636576.0
233233289939.89682
59
+------+------+------+------+------+
|      |      |      |      |  2   |
+------+------+------+------+------+
| 102

76
+------+------+------+------+------+
|  2   |  8   |      |      |      |
+------+------+------+------+------+
|  32  |  64  |  4   |  32  |      |
+------+------+------+------+------+
|  2   | 2048 |  4   |      |      |
+------+------+------+------+------+
|  8   |  4   | 128  |  32  |  2   |
+------+------+------+------+------+
|  2   | 256  |  8   |  2   |  64  |
+------+------+------+------+------+
1
4287748.0
173648172275.6904
77
+------+------+------+------+------+
| 2048 | 512  |      | 512  |      |
+------+------+------+------+------+
|  2   |  4   |  32  |  4   |      |
+------+------+------+------+------+
|  8   |  4   |  16  |  4   |  8   |
+------+------+------+------+------+
|  16  |  32  | 256  | 1024 | 256  |
+------+------+------+------+------+
|  2   |  8   |  64  |  2   |      |
+------+------+------+------+------+
1
5905164.0
364320682548.40356
78
+------+------+------+------+------+
|  2   |      |  2   |  32  |  2   |
+------+------+------+------+------+
|  4 

95
+------+------+------+------+------+
| 128  |  2   |  16  |      |      |
+------+------+------+------+------+
| 256  |  8   |  2   |  4   |      |
+------+------+------+------+------+
| 128  |  2   |  32  |  64  | 1024 |
+------+------+------+------+------+
|  16  | 512  |  2   |  8   |  2   |
+------+------+------+------+------+
| 128  |  8   |  32  |  2   | 2048 |
+------+------+------+------+------+
1
5626600.0
93633842547.97923
96
+------+------+------+------+------+
|  4   |  4   |  4   |  8   |  4   |
+------+------+------+------+------+
| 2048 |  2   | 256  |  32  |  8   |
+------+------+------+------+------+
|      |  32  |  4   | 256  |  32  |
+------+------+------+------+------+
|      |  8   |      |  16  |  2   |
+------+------+------+------+------+
|      |  2   |      |      | 128  |
+------+------+------+------+------+
1
4345372.0
66394906750.938446
97
+------+------+------+------+------+
|      |  2   |  8   | 256  |  8   |
+------+------+------+------+------+
|    

In [11]:
f = open("res2.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

In [5]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [ ]:
 time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+
|  16  |  8   |  64  |  2   |
+------+------+------+------+
|  2   |  32  | 512  | 256  |
+------+------+------+------+
|  16  | 128  |  32  |  8   |
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
2
350896.0
209399003412.29755
1
+------+------+------+------+
|  2   |  16  |  8   |  4   |
+------+------+------+------+
|  8   |  32  |  4   |  2   |
+------+------+------+------+
|  32  |  8   |  16  |  64  |
+------+------+------+------+
|  8   | 256  |  4   |  2   |
+------+------+------+------+
2
72508.0
154921215497.19693
2
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
|  8   | 256  |  8   |  16  |
+------+------+------+------+
|  2   |  32  | 128  |  2   |
+------+------+------+------+
|  4   |  64  |  8   |  4   |
+------+------+------+------+
2
87820.0
156623647610.9081
3
+------+------+------+------+
|  2   |  4   |  32  |  2   |
+------+------+------+------+
|  64

# TEST 3 - Rand Walk

# TEST 4 - How High

In [12]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
                
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        #print(self.unexplored_moves)
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        #print(leaf.grid)
        sim_res = rollout_o(leaf, start_grid)
        #print(sim_res)
        backpropagate(leaf, sim_res)
        #print(root.children)

    # When done iterating, return the 'best' child of the root node.\
    try:
        return best_child(root).move
    except:
        #print(root.children)
        return root

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        #print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)
    
    # If the node is terminal, return it
    if node.is_terminal_state:
        #print(1)
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    return sample.board_value()/(16384 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [ ]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

8388608.0
+------+------+------+------+------+
|  4   |  8   |8388608|  2   |4194304|
+------+------+------+------+------+
|  8   |  32  |2097152|  16  |  4   |
+------+------+------+------+------+
|  2   | 128  |  64  |131072|1048576|
+------+------+------+------+------+
|  8   |  4   | 1024 |  16  |  8   |
+------+------+------+------+------+
|  4   |  2   |  4   | 128  |  2   |
+------+------+------+------+------+
2
93475669317472.0
737157286712.4755
